### 1. 필요 라이브러리 import

pandas, numpy 불러오기

In [ ]:
import pandas as pd
import numpy as np

num_to_label pickle 에서 불러오려면 아래 셀 uncomment 후 경로 지정

In [ ]:
# import pickle
# with open("./path/to/dict_num_to_label.pkl",'rb') as fr:
#     num_to_label = pickle.load(fr)
# with open("./path/to/dict_label_to_num.pkl",'rb') as fr:
#     label_to_num = pickle.load(fr)
# print(num_to_label)
# print(label_to_num)

num_to_label, label_to_num dict 직접 만들기

In [ ]:
num_to_label = {0: 'no_relation', 1: 'org:top_members/employees', 2: 'org:members', 3: 'org:product', 4: 'per:title', 5: 'org:alternate_names', 6: 'per:employee_of', 7: 'org:place_of_headquarters', 8: 'per:product', 9: 'org:number_of_employees/members', 10: 'per:children', 11: 'per:place_of_residence', 12: 'per:alternate_names', 13: 'per:other_family', 14: 'per:colleagues', 15: 'per:origin', 16: 'per:siblings', 17: 'per:spouse', 18: 'org:founded', 19: 'org:political/religious_affiliation', 20: 'org:member_of', 21: 'per:parents', 22: 'org:dissolved', 23: 'per:schools_attended', 24: 'per:date_of_death', 25: 'per:date_of_birth', 26: 'per:place_of_birth', 27: 'per:place_of_death', 28: 'org:founded_by', 29: 'per:religion'}
label_to_num = {'no_relation': 0, 'org:top_members/employees': 1, 'org:members': 2, 'org:product': 3, 'per:title': 4, 'org:alternate_names': 5, 'per:employee_of': 6, 'org:place_of_headquarters': 7, 'per:product': 8, 'org:number_of_employees/members': 9, 'per:children': 10, 'per:place_of_residence': 11, 'per:alternate_names': 12, 'per:other_family': 13, 'per:colleagues': 14, 'per:origin': 15, 'per:siblings': 16, 'per:spouse': 17, 'org:founded': 18, 'org:political/religious_affiliation': 19, 'org:member_of': 20, 'per:parents': 21, 'org:dissolved': 22, 'per:schools_attended': 23, 'per:date_of_death': 24, 'per:date_of_birth': 25, 'per:place_of_birth': 26, 'per:place_of_death': 27, 'org:founded_by': 28, 'per:religion': 29}

### 2. 앙상블 재료들 DataFrame으로 불러오기

In [ ]:
# 본인이 앙상블할 재료 submission.csv들로 경로 변경 (ex. 점수 높은 순서대로 1위, 2위, 3위 => 각자 3, 2, 1 표 행사)
ens1 = pd.read_csv('../submission/sub_65.csv')
ens2 = pd.read_csv('../submission/sub_62.csv')
ens3 = pd.read_csv('../submission/sub_57.csv')
ens4 = pd.read_csv('../submission/sub_52.csv')
# 리스트로 선언
ens_list = [ens1, ens2, ens3, ens4]


In [ ]:
# 잘 import 되었나 길이 확인
for idx, ens in enumerate(ens_list):
    print(f"ens{idx + 1}의 길이 : {len(ens)}")

### 3. 확률 계산을 위해 'probs' 칼럼 float 리스트로 만들어서 새 칼럼에 추가

In [ ]:
for ens in ens_list:
    ens['probs_float'] = ens['probs'].apply(lambda x: list(map(float, x[1:-1].split(', '))))

### 4. 예측 라벨 index와 확률을 튜플 형태로 가진 pred_idx_prob 칼럼을 새로 만듦

In [ ]:
for ens in ens_list:
    ens['pred_idx_prob'] = ens['pred_label']
    for i in range(len(ens)):
        label_idx = label_to_num[ens['pred_label'][i]]
        ens['pred_idx_prob'][i] = (label_idx, ens['probs_float'][i][label_idx])
    

pred_idx_prob 칼럼 포함 결과물 출력해보기

In [ ]:
ens1.head()

### 5. 앙상블 출력 파일용 뼈대 ens_result 만들기

In [ ]:
ens_result = ens1.copy()
# ens_result = ens1.copy().drop(columns=['pred_idx_prob','probs_float'])

### 6. Hard Voting (다수결) 알고리즘

In [ ]:
# weight_list = [2.0, 1.0, 1.0, 1.0]
same_value_count = 0

for i in range(len(ens_result)):
    voting_dict = {i:0 for i in range(30)}
    for idx, ens in enumerate(ens_list):
        voting_dict[ens['pred_idx_prob'][i][0]] += len(ens_list) - idx # 순위대로 4, 3, 2, 1... 가중치 부여
        # voting_dict[ens['pred_idx_prob'][i][0]] += 1 # 모두 공평하게 1씩 더하기
        # voting_dict[ens['pred_idx_prob'][i][0]] += weight_list[idx] # weight_list 에서 선언한 임의의 가중치로 더하기
    top_label = max(voting_dict, key=voting_dict.get) # 가장 높은 voting을 받은 label
    ens_result['pred_label'][i] = num_to_label[top_label]

각 모델과 다르게 예측한 pred_label의 개수 확인


In [ ]:
# counts = []
for ens, model_num in zip(ens_list, range(1, len(ens_list) + 1)):
    result_comparison = ens_result['pred_label'] != ens['pred_label']
    count_true = result_comparison.sum()
    # counts.append(count_true)
    print(f"모델 {model_num} 과 다른 예측 개수 : {count_true}")


### 7. probs 칼럼은 산술평균치로 계산해주기 (auprc 점수를 위해)

In [ ]:
# probs도 단순 산술평균치로 대체
for i in range(len(ens_result)):
    temp_arr = []
    for ens in ens_list:
        temp_arr.append(ens['probs_float'][i])
    ens_result['probs_float'][i] = np.mean(temp_arr, axis=0)

# 기존 probs 칼럼을 평균된 probs_float 값으로 바꿔주기
ens_result['probs'] = ens_result['probs_float'].apply(lambda x: '[' + ', '.join(map(str, x)) + ']')

### 8. 바로 채점하기

In [ ]:
preds = list(ens_result['pred_label'])
labels = list(pd.read_csv('./klue-dev/klue_dev_answer_sorted.csv')['label'])
label_list = ['org:top_members/employees', 'org:members',
    'org:product', 'per:title', 'org:alternate_names',
    'per:employee_of', 'org:place_of_headquarters', 'per:product',
    'org:number_of_employees/members', 'per:children',
    'per:place_of_residence', 'per:alternate_names',
    'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
    'per:spouse', 'org:founded', 'org:political/religious_affiliation',
    'org:member_of', 'per:parents', 'org:dissolved',
    'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
    'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
    'per:religion']

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1, none = precision_recall_fscore_support(labels, preds, average='micro', labels=label_list)
print(f"F1: {f1 * 100}\nPrecision: {precision * 100}\nRecall: {recall * 100}")

### 8. csv 파일로 생성하기

In [ ]:
ens_result = ens_result[['id','pred_label','probs']]
# ens_result.head()

In [ ]:
import datetime
ts = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
ens_result.to_csv(f'../submission/{ts}_weighted_hard_voting.csv', index=False)